Embedding: Tutoriel 01 

In [66]:
import torch 
import torch.nn as nn #For load model
import torch.nn.functional as F #For having acces to all functions available in pytorch hub
import torch.optim as optim  #For optimization

In [19]:
CONTEXT_SIZE = 3 #Given 2 word we should predict the next word 
EMBEDDING_DIM = 10 #For represent each word inside the text in to a 10 dimentional space 

In [58]:
text = """
Docteur : Bonjour, comment ça va aujourd'hui ?

Patient : Bonjour, j'ai des douleurs dans le bas-ventre depuis quelques semaines maintenant et je suis très inquiète.

Docteur : Je vois. Pouvez-vous me dire où exactement vous ressentez ces douleurs ?

Patient : C'est principalement dans la région ovarienne, mais ça peut aussi irradier dans le dos.

Docteur : D'accord. Est-ce que vous avez remarqué d'autres symptômes comme des saignements anormaux ou des pertes vaginales inhabituelles ?

Patient : Oui, j'ai remarqué que mes règles étaient plus abondantes et plus longues que d'habitude. Et j'ai aussi des pertes vaginales un peu plus abondantes.

Docteur : Je comprends. Avez-vous remarqué si ces symptômes sont liés à votre cycle menstruel ou s'ils se produisent de manière aléatoire ?

Patient : C'est difficile à dire, mais je pense que ça se produit de manière aléatoire.

Docteur : Très bien. Nous allons devoir faire des examens pour en savoir plus sur ce qui se passe. Il est possible que nous ayons besoin de faire une échographie ou une biopsie pour mieux comprendre la situation.

Patient : D'accord, je comprends. Mais ça ne peut pas être quelque chose de grave, n'est-ce pas ?

Docteur : Il est difficile de dire pour le moment. Nous devons d'abord faire des examens pour en savoir plus. Mais ne vous inquiétez pas, nous allons tout faire pour trouver la cause de vos symptômes et vous aider à vous sentir mieux.
"""

In [59]:
text = text.replace("Patient :" , "")
text = text.replace("Docteur :" , "")
text = text.strip()
sliceText = text.split()

N-GRAM

It is a nlp technique used to slice a text in N succession of word. 
eg : text = papa est sorti voir maman au village 
2 grams:
papa est 
est sorti
sorti voir 
voir maman
maman au 
au village

In [50]:

# Reversing a list using slicing technique
def Reverse(lst):
    new_lst = lst[::-1]
    return new_lst

In [51]:
ngrams = [
    (
        Reverse([sliceText[i - j - 1] for j in range(CONTEXT_SIZE)]),
        sliceText[i]
    )
    for i in range(CONTEXT_SIZE, len(sliceText))
]

Vocabulary 

In [60]:
vocab = set(sliceText)

In [64]:
ids = {word : idx for idx , word in enumerate(vocab)}

In [67]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [68]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [70]:

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([ids[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([ids[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[1060.3521480560303, 1054.8881106376648, 1049.4772410392761, 1044.1156001091003, 1038.7993278503418, 1033.5264983177185, 1028.288739681244, 1023.0860347747803, 1017.9147090911865, 1012.7678003311157]


In [72]:
# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[ids["remarqué"]])

tensor([-1.2569, -0.3631, -0.0493, -0.5748,  0.0583,  0.2807, -0.4966, -2.1175,
         0.6554,  0.2405], grad_fn=<SelectBackward0>)
